In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
# load local version of gallifrey, before pip installed version, for debugging
import pathlib
import sys
import os

sys.path.append(str(pathlib.Path(os.getcwd()).parent.joinpath("src")))

In [ ]:
import yt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from gallifrey.setup import data_setup
from gallifrey.utilities.math import calculate_pca
from gallifrey.utilities.dataframe import aggregated_dataframe, rename_labels
from gallifrey.visualization.visualization_utils import (
    set_plot_defaults,
    get_palette,
    FigureProcessor,
    ridgeplot,
)

## DATA SETUP

In [ ]:
snapshot = 127
resolution = 8192
sim_id = "37_11"

In [ ]:
ds, mw, stellar_model, imf, planet_model, path = data_setup(
    snapshot=snapshot,
    resolution=resolution,
    sim_id=sim_id,
)

## PLOT SETUP

In [ ]:
set_plot_defaults()

figure_directory = (
    f"MilkyWayAnalogue/{resolution}/{sim_id}/{snapshot}/Galaxy_components"
)
save = True

## CREATE DATA SOURCE

In [ ]:
radius = 0.2 * mw.virial_radius()
sphere_data = mw.sphere(radius=radius)

In [ ]:
components = ["bulge_stars", "thin_disk_stars", "thick_disk_stars", "halo_stars"]
data = aggregated_dataframe(
    components,
    [
        "[Fe/H]",
        "stellar_age",
        "Masses",
    ],
    data_source=sphere_data,
    type_name="Component",
    custom_units={"stellar_age": "Gyr"},
)

# rename components
data["Component"] = data["Component"].apply(
    lambda type: type.replace("_", " ").replace(" stars", "").title()
    if type.endswith("_stars")
    else type
)

# filtering values
filters = {"[Fe/H]": (-2.5, 1)}
for field_value, ranges in filters.items():
    data = data[data[field_value].between(*ranges)]

# rename labels
data, labels = rename_labels(data)

## FRACTION OF MASS IN COMPONENTS

In [ ]:
data.groupby("Component")["Masses"].sum()

In [ ]:
data.groupby("Component")["Masses"].sum() / data["Masses"].sum()

## STELLAR AGE IN COMPONENTS

In [ ]:
data.groupby("Component")[labels["stellar_age"]].describe().T

In [ ]:
ridgeplot_age = ridgeplot(
    data,
    x=labels["stellar_age"],
    row="Component",
    palette=get_palette(n_colors=4),
    label_position=[0.98, 0.22],
)
FigureProcessor(ridgeplot_age.figure).save(
    "ridgeplot_age.pdf",
    sub_directory=figure_directory,
    save=save,
)

## METALLICITY IN COMPONENTS

In [ ]:
data.groupby("Component")["[Fe/H]"].describe().T

In [ ]:
ridgeplot_metallicity = ridgeplot(
    data,
    x=labels["[Fe/H]"],
    row="Component",
    palette=get_palette(n_colors=4),
    label_position=[0.34, 0.22],
)
FigureProcessor(ridgeplot_metallicity.figure).save(
    "ridgeplot_metallicity.pdf",
    sub_directory=figure_directory,
    save=save,
)

## PRICIPAL COMPONENTS

In [ ]:
variance_ratios = pd.DataFrame(
    np.array(
        [
            calculate_pca(
                sphere_data[component, "particle_position"].to("kpc")
            ).explained_variance_ratio_
            for component in components
        ]
    ).T,
    columns=components,
).T

print("Minor-to-major axis ratio:")
print(variance_ratios[2] / variance_ratios[0])